# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import pandas as pd
import numpy as np

In [6]:
# Parse XML file to read
document = ET.parse( './data/mondial_database.xml' )

## 1. 10 countries with the lowest infant mortality rates

In [7]:
# Create a dictionary for country/mortality rate
countries = {}

# Loop over each country, pass those without name/rate
for child in document.iterfind('country'):
    try:
        country = child.find('name')
        infant = child.find('infant_mortality')
        countries[country.text] = float(infant.text)
    except AttributeError:
        pass

# Create dataframe
df_1 = pd.DataFrame.from_dict(countries, orient = 'index').reset_index()
df_1.columns = ["Country", "Infant mortality rate"]
df_1 = df_1.sort_values(by='Infant mortality rate').head(10)
df_1 = df_1.reset_index().drop('index', 1)
df_1

,Country,Infant mortality rate
0,Monaco,1.81
1,Japan,2.13
2,Bermuda,2.48
3,Norway,2.48
4,Singapore,2.53
5,Sweden,2.60
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


## 2. 10 cities with the largest population

In [8]:
# Make a list for adding up population
population_list = []

# Loop over each country, city, and province
for country in document.iterfind('country'):
    cities = country.findall('city')
    provinces = country.findall('province')
    # Adding provinces
    for province in provinces:
        cities += province.findall('city')
    for city in cities:
        city_name = city.find('name').text
        population = city.find('.//population[last()]')
        if population != None:
            city_pop = int(population.text)
        recent_pop = (country.find('name').text,
                      city_name, city_pop)
        population_list.append(recent_pop)

# Create dataframe
labels = ['Country', 'City', 'Population']
df_2 = pd.DataFrame.from_records(population_list, columns=labels)
df_2 = df_2.sort_values('Population', ascending=False)
df_2 = df_2.head(10).reset_index().drop('index', 1)
df_2

,Country,City,Population
0,China,Shanghai,22315474
1,Turkey,Istanbul,13710512
2,India,Mumbai,12442373
3,Russia,Moskva,11979529
4,China,Beijing,11716620
5,Brazil,São Paulo,11152344
6,China,Tianjin,11090314
7,China,Guangzhou,11071424
8,India,Delhi,11034555
9,China,Shenzhen,10358381


## 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [9]:
# Create a dictionary for country and population
country_pop = {}

# Finding most recent population
for element in document.iterfind('country'):
    country = element.find('name')
    population = element.find('.//population[last()]')
    if (country) != None:
        if (population) != None:
            country_pop[country.text] = int(population.text)
    else:
        country_pop[country.text] = np.NaN
ctry_df = pd.DataFrame.from_dict(country_pop, orient='index')
ctry_df.reset_index(drop=False, inplace=True)
ctry_df.columns = ['Country', 'Population']

# Create a dictionary for ethnicities
ethnicity = []
ethnicity_labels = ['Country', 'Ethnicity']

# Finding ethnic groups
for country in document.iterfind('country'):
    ctry = country.find('name').text
    for ethnic in country.iterfind('ethnicgroup'):
        ethnic_name = ethnic.text
        ethnicity.append([ctry, ethnic_name])
ethnicity_df = pd.DataFrame(ethnicity, columns=ethnicity_labels)
ethnicity_df = ethnicity_df.merge(
    ctry_df[['Country', 'Population']], on='Country')

# Create dataframe
df_3 = ethnicity_df.groupby('Ethnicity').sum()
df_3.sort_values(by='Population', ascending=False, inplace=True)
df_3.head(10).reset_index()

,Ethnicity,Population
0,Han Chinese,1360720000
1,Mongol,1213609662
2,Indo-Aryan,1210854977
3,Dravidian,1210854977
4,European,1157295639
5,African,975352746
6,Amerindian,588752467
7,Malay,377500275
8,Asian,374650120
9,Russian,322438406


## 4. Name and country of: a) longest river, b) largest lake, and c) airport at highest elevation

In [10]:
# Using collections to save time sorting and indexing.
import collections

In [11]:
# Create a dictionary for countries
countries = {}
for child in document.getiterator('country'):
    try:
        countries[child.attrib['car_code']] = child.find('name').text 
    except AttributeError:
        pass
countries = collections.OrderedDict(sorted(countries.items()))

# Create a dictionary for rivers
longest_river = {}
temp_len = 0

for elt in document.getiterator('river'):
    try:
        if float(elt.find('length').text) > temp_len:
            temp_len = float(elt.find('length').text)
            temp_coun = elt.attrib['country']
            temp_name = elt.find('name').text
    except AttributeError:
        pass

# Create data frame
longest_river[temp_name] = countries[temp_coun.split(' ')[0]]
df_river = pd.DataFrame.from_dict(longest_river, orient='index').reset_index()
df_river.columns = ['River Name', 'Country']
df_river

,River Name,Country
0,Amazonas,Colombia


In [12]:
# Create a dictionary for lakes
largest_lake = {}
temp_volume = 0
max_volume = 0
for elt in document.getiterator('lake'):
    try:
        temp_volume = float(elt.find('area').text) * float(elt.find('depth').text)  #to find largest lake = area*depth
        if temp_volume > max_volume:
            max_volume = temp_volume
            temp_coun = elt.attrib['country']
            temp_name = elt.find('name').text
    except AttributeError:
        pass

# Create data frame
largest_lake[temp_name] = countries[temp_coun.split(' ')[0]]
df_lake = pd.DataFrame.from_dict(largest_lake, orient='index').reset_index()
df_lake.columns = ['Largest Lake', 'Country']
df_lake

,Largest Lake,Country
0,Caspian Sea,Russia


In [13]:
# Create a dictionary for airports
highest_airport = {}
temp_high = 0
for elt in document.getiterator('airport'):
    try:
        if float(elt.find('elevation').text) > temp_high:
            temp_high = float(elt.find('elevation').text)
            temp_coun = elt.attrib['country']
            temp_name = elt.find('name').text
    except AttributeError:
        pass
    except TypeError:
        pass

# Create data frame
highest_airport[temp_name] = countries[temp_coun.split(' ')[0]]
df_airport = pd.DataFrame.from_dict(highest_airport, orient='index').reset_index()
df_airport.columns = ['Highest Airport', 'Country']
df_airport

,Highest Airport,Country
0,El Alto Intl,Bolivia
